In [1]:
import numpy as np
import json
import glob
import pandas as pd
import csv

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy
import spacy
from nltk.corpus import stopwords

# Vis
import pyLDAvis
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
all_files = ["robinhood.csv", "webull.csv", "etoro.csv"]

In [3]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = pd.read_csv(file)
        # print(data)
        # data = json.load(f)
    return (data);

def write_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4);

# Combine all the csv files into one

li = []
for filename in all_files:
    df = load_data(filename)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
data = frame.loc[:, ["content", "score"]]
data = data.loc[(data.score <= 2)] # Grab only poor reviews
content = json.loads(data.loc[:, "content"].to_json())

# Prefilter step

# Remove reviews with <= 50 chars
data = [content[key] for key in content if len(content[key]) > 50];
len(data)



3153

In [4]:
# Stop words are basically words that are not really useful in the topic model
# i.e. words that you want to eliminate from consideration
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# data = load_data("ushmm_dn.json")["texts"]
print(data[0][0:90]) # Prints out first text, char 0-89

I once wrote my reviews here, how robinhood had been messing with my funds in my account, 


In [6]:
# Reducing words to their root form (lemma) using spacy or nltk to grab roots
# Specify allowed_postags for spacy

# Takes a list of texts i.e. [text1, text2...] and returns lemmatised [text1..]
# Allowed_post_tages -> Basically default arguments in python 
# This is a list initalised that is used to filter tokens 
# pos_ tag marks category of word token belongs to i.e. only append nouns,adj,verbs,and adverbs 
# e.g. conjunctions (but, and, because), and pronouns (he/she) is ignored
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):

    # Spacy load creates an an nlp parser i.e. function pointer 
    spacy.load
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []

    for text in texts:
        doc = nlp(text) # Create document from the raw text/string
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags and token.pos_ not in stopwords:
               new_text.append(token.lemma_) # Appends only lemma to new_text
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out);

lemmatized_texts = lemmatization(data)
print(lemmatized_texts[0][0:90])

once write review here robinhood mess fund account very easy transfer money robinhood link


In [7]:
# Simple preprocess converts documents to list of lower case tokens ignoring ones
# which are too short or too long.
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True);
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])


['once', 'write', 'review', 'here', 'robinhood', 'mess', 'fund', 'account', 'very', 'easy', 'transfer', 'money', 'robinhood', 'link', 'account', 'never', 'go', 'easy', 'take', 'money']


In [8]:
# BIGRAMS AND TRIGRAMS

# min_count - ignore all words/bigrams with total collected count lower than this value.
# threshold - score for forming phrases play around with this -> related to scoring function
bigrams_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=1) 

# First arg checks ensures no overlap between bigraph phrases and trigram phrases.
trigram_phrases = gensim.models.Phrases(bigrams_phrases[data_words], threshold=1)

# Create function pointer i.e. phraser that converts document to list of bigram
bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# print(data_bigrams_trigrams[0])

# new_words = []
# for word in data_bigrams_trigrams[0]:
#     if '_' in word:
#         new_words.append(word)
# print(data_bigrams)


In [9]:
# TF-IDF Removal - to remove words that are not important to the topic
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

# Goes through words and looks at words that occur too frequentely and removes them
# Corpus becomes new corpus where those irrelevant words are removed
low_value = 0.03
words = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow




[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [10]:
# id2word = corpora.Dictionary(data_words)
# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text) # Create a bag of words all (word: f pairs) for 1 doc
#     corpus.append(new)

# print(corpus[0][0:20])
# #(0,2) - 0 = index of word in the dictionary, 2 = frequency of word in that text

# word = id2word[[0][:1][0]] # i.e. text 1 -> first tuple -> first value in tuple
# print(word) # i.e. able is the first element in teh dictionary 

In [11]:
lda_model = gensim.models.LdaModel(corpus=corpus
                                    ,id2word=id2word
                                    ,num_topics=10
                                    ,random_state=100
                                    ,update_every=1
                                    ,chunksize=100
                                    ,passes=10
                                    ,alpha="auto")

### Idea
I think I can separate the review data set into one star vs 5 star reviews and look at if there are any differences in the topics that are mentioned.

In [12]:
# Visualising the data
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/root/miniconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: Deprecatio

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.442890 -0.118910       1        1  26.071629
1     -0.101767  0.411978       2        1  18.532779
7     -0.348493  0.236953       3        1  15.732906
8      0.199454 -0.198314       4        1   6.988090
3      0.405617 -0.097707       5        1   6.963278
2      0.336413  0.172645       6        1   6.781559
9      0.147275  0.292901       7        1   5.177110
5      0.071648 -0.389598       8        1   5.080534
0     -0.102950 -0.008835       9        1   4.435704
6     -0.164306 -0.301113      10        1   4.236410, topic_info=                  Term        Freq       Total Category  logprob  loglift
68                 app  786.000000  786.000000  Default  30.0000  30.0000
294   customer_service  257.000000  257.000000  Default  29.0000  29.0000
98                 fee  201.000000  201.000000  Default  28.0000  28.0000
4003             etoro  321.000000  321.000000  Default  27.0000  27.0000
16               money  445.000000  445.000000  Default  26.0000  26.0000
...                ...         ...         ...      ...      ...      ...
727              trash   18.502035   19.396952  Topic10  -4.7884   3.1142
1035           respond   18.475023   19.369913  Topic10  -4.7898   3.1142
3667             focus   18.308470   19.206362  Topic10  -4.7989   3.1136
703              limit   18.247316   19.142274  Topic10  -4.8022   3.1136
1142    bank_statement   17.450520   18.345506  Topic10  -4.8469   3.1114

[346 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
729       4  0.984841          accept
953       6  0.981727          access
105       1  0.504279         account
105       2  0.494092         account
935       3  0.986908              ad
...     ...       ...             ...
1189      5  0.987596  withdraw_money
748       3  0.990007      withdrawal
64        3  0.992862            work
1759      6  0.977322           wrong
617       7  0.984915             yet

[332 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 8, 9, 4, 3, 10, 6, 1, 7])

In [13]:
# Save the model into memory
lda_model.save(f"models/{csv_file[:-4]}.model");

NameError: name 'csv_file' is not defined